In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
from tqdm import tqdm
import gc
import dask
from dask import dataframe as dd
from dask.diagnostics import ProgressBar

In [2]:
# To wrap long text lines
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [3]:
# For fancy table Display
# %load_ext google.colab.data_table

dask.config.set(num_workers=8, scheduler='processes')

tqdm.pandas()

In [4]:
# Load the data
metadata_df = pd.read_csv('./data/metadata.csv', low_memory=False)
metadata_df

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544401,x7mhzhw9,1fca7a77b04fc4a1b326c8a48964ee0d6a768e86,Medline; PMC,The COVID19 pandemic crisis and the relevance ...,10.1007/s12571-020-01071-6,PMC7343897,32837643,cc-by,The Covid19 pandemic should be seen as a wake-...,2020-07-09,"Bhavani, R. V.; Gopinath, R.",Food Secur,NaN,NaN,NaN,document_parses/pdf_json/1fca7a77b04fc4a1b326c...,document_parses/pmc_json/PMC7343897.xml.json,https://doi.org/10.1007/s12571-020-01071-6; ht...,220417712.0
544402,n47e9u3c,91351188e43719fe66f83b9144832073eae26dc9,Elsevier; PMC,Law and policy of platform economy in China,10.1016/j.clsr.2020.105493,PMC7581424,NaN,els-covid,China is experiencing a phenomenal expansion o...,2020-11-30,"You, Chuanman",Computer Law & Security Review,NaN,NaN,NaN,document_parses/pdf_json/91351188e43719fe66f83...,NaN,https://www.sciencedirect.com/science/article/...,225041999.0
544403,yia4uww0,612fef2d60ed24807bf9ce083bc619059885d1dd,Medline; PMC,The perception of nurses towards their roles d...,10.1111/ijcp.13919,PMC7883258,33296522,no-cc,PURPOSE: The study aims to evaluate the availa...,2020-12-23,"Abuhammad, Sawsan; AlAzzam, Manar; Mukattash, ...",Int J Clin Pract,NaN,NaN,NaN,document_parses/pdf_json/612fef2d60ed24807bf9c...,document_parses/pmc_json/PMC7883258.xml.json,https://doi.org/10.1111/ijcp.13919; https://ww...,228089464.0
544404,nrpiyq1g,645c9aec6eed5f1c1436ab4083d35f1107e6467e,Medline; PMC,Choroid Plexus: The Orchestrator of Long-Range...,10.3390/ijms21134760,PMC7369786,32635478,cc-by,Cerebrospinal fluid (CSF) is the liquid that f...,2020-07-04,"Kaiser, Karol; Bryja, Vitezslav",Int J Mol Sci,NaN,NaN,NaN,document_parses/pdf_json/645c9aec6eed5f1c1436a...,document_parses/pmc_js

In [5]:
# Explore the metadata
metadata_df.info()
metadata_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544406 entries, 0 to 544405
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   cord_uid          544406 non-null  object 
 1   sha               183834 non-null  object 
 2   source_x          544406 non-null  object 
 3   title             544158 non-null  object 
 4   doi               298986 non-null  object 
 5   pmcid             193683 non-null  object 
 6   pubmed_id         258178 non-null  object 
 7   license           544406 non-null  object 
 8   abstract          396056 non-null  object 
 9   publish_time      544187 non-null  object 
 10  authors           529539 non-null  object 
 11  journal           508745 non-null  object 
 12  mag_id            0 non-null       float64
 13  who_covidence_id  226293 non-null  object 
 14  arxiv_id          7097 non-null    object 
 15  pdf_json_files    183834 non-null  object 
 16  pmc_json_files    14

,mag_id,s2_id
count,0.0,4.952350e+05
mean,NaN,1.976635e+08
std,NaN,6.753983e+07
min,NaN,9.600000e+01
25%,NaN,2.186196e+08
50%,NaN,2.213713e+08
75%,NaN,2.281010e+08
max,NaN,2.333898e+08


In [6]:
# Drop rows with null abstract
metadata_df = metadata_df.loc[~pd.isna(metadata_df['abstract'])]
metadata_df

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544401,x7mhzhw9,1fca7a77b04fc4a1b326c8a48964ee0d6a768e86,Medline; PMC,The COVID19 pandemic crisis and the relevance ...,10.1007/s12571-020-01071-6,PMC7343897,32837643,cc-by,The Covid19 pandemic should be seen as a wake-...,2020-07-09,"Bhavani, R. V.; Gopinath, R.",Food Secur,NaN,NaN,NaN,document_parses/pdf_json/1fca7a77b04fc4a1b326c...,document_parses/pmc_json/PMC7343897.xml.json,https://doi.org/10.1007/s12571-020-01071-6; ht...,220417712.0
544402,n47e9u3c,91351188e43719fe66f83b9144832073eae26dc9,Elsevier; PMC,Law and policy of platform economy in China,10.1016/j.clsr.2020.105493,PMC7581424,NaN,els-covid,China is experiencing a phenomenal expansion o...,2020-11-30,"You, Chuanman",Computer Law & Security Review,NaN,NaN,NaN,document_parses/pdf_json/91351188e43719fe66f83...,NaN,https://www.sciencedirect.com/science/article/...,225041999.0
544403,yia4uww0,612fef2d60ed24807bf9ce083bc619059885d1dd,Medline; PMC,The perception of nurses towards their roles d...,10.1111/ijcp.13919,PMC7883258,33296522,no-cc,PURPOSE: The study aims to evaluate the availa...,2020-12-23,"Abuhammad, Sawsan; AlAzzam, Manar; Mukattash, ...",Int J Clin Pract,NaN,NaN,NaN,document_parses/pdf_json/612fef2d60ed24807bf9c...,document_parses/pmc_json/PMC7883258.xml.json,https://doi.org/10.1111/ijcp.13919; https://ww...,228089464.0
544404,nrpiyq1g,645c9aec6eed5f1c1436ab4083d35f1107e6467e,Medline; PMC,Choroid Plexus: The Orchestrator of Long-Range...,10.3390/ijms21134760,PMC7369786,32635478,cc-by,Cerebrospinal fluid (CSF) is the liquid that f...,2020-07-04,"Kaiser, Karol; Bryja, Vitezslav",Int J Mol Sci,NaN,NaN,NaN,document_parses/pdf_json/645c9aec6eed5f1c1436a...,document_parses/pmc_js

In [7]:
nlp = spacy.load('en_core_web_md')

# Tokenize and lemmatize the data
metadata_df['lemmas'] = metadata_df['abstract'].progress_apply(lambda abstract: [tk.lemma_ for tk in nlp(abstract)])
metadata_df

100%|██████████| 396056/396056 [3:06:34<00:00, 35.38it/s]  
<ipython-input-7-35229c5820b5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_df['lemmas'] = metadata_df['abstract'].progress_apply(lambda abstract: [tk.lemma_ for tk in nlp(abstract)])


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,lemmas
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,"[objective, :, this, retrospective, chart, rev..."
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[inflammatory, disease, of, the, respiratory, ..."
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[surfactant, protein, -, d, (, SP, -, D, ), pa..."
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[Endothelin-1, (, et-1, ), be, a, 21, amino, a..."
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"[respiratory, syncytial, virus, (, RSV, ), and..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544401,x7mhzhw9,1fca7a77b04fc4a1b326c8a48964ee0d6a768e86,Medline; PMC,The COVID19 pandemic crisis and the relevance ...,10.1007/s12571-020-01071-6,PMC7343897,32837643,cc-by,The Covid19 pandemic should be seen as a wake-...,2020-07-09,"Bhavani, R. V.; Gopinath, R.",Food Secur,NaN,NaN,NaN,document_parses/pdf_json/1fca7a77b04fc4a1b326c...,document_parses/pmc_json/PMC7343897.xml.json,https://doi.org/10.1007/s12571-020-01071-6; ht...,220417712.0,"[the, Covid19, pandemic, should, be, see, as, ..."
544402,n47e9u3c,91351188e43719fe66f83b9144832073eae26dc9,Elsevier; PMC,Law and policy of platform economy in China,10.1016/j.clsr.2020.105493,PMC7581424,NaN,els-covid,China is experiencing a phenomenal expansion o...,2020-11-30,"You, Chuanman",Computer Law & Security Review,NaN,NaN,NaN,document_parses/pdf_json/91351188e43719fe66f83...,NaN,https://www.sciencedirect.com/science/article/...,225041999.0,"[China, be, experience, a, phenomenal, expansi..."
544403,yia4uww0,612fef2d60ed24807bf9ce083bc619059885d1dd,Medline; PMC,The perception of nurses towards their roles d...,10.1111/ijcp.13919,PMC7883258,33296522,no-cc,PURPOSE: The study aims to evaluate the availa...,2020-12-23,"Abuhammad, Sawsan; AlAzzam, Manar; Mukattash, ...",Int J Clin Pract,NaN,NaN,NaN,document_parses/pdf_json/612fef2d60ed24807bf9c...,document_parses/pmc_json/PMC7883258.xml.json,https://doi.org/10.1111/ijcp.13919; https://ww.

In [8]:
# Save the preprocessed data
metadata_df.to_csv('./data/metadata_processed.csv', index=False)
nlp.to_disk('./data/en_core_extended')

In [ ]:
# @dask.delayed
# def lemmatize(abstract, nlp):
#     '''
    
#     '''
#     doc = nlp(abstract)
#     lemmas = [tk.lemma_ for tk in doc]
#     return lemmas

# results = []
# for idx, row in metadata_df.iterrows():
#     d = dask.delayed(lemmatize)(row['abstract'], nlp)
#     results.append(d)
    

# with ProgressBar():
#     results = dask.compute(*results)
    
# metadata_df['lemmas'] = results
# metadata_df
    
def lemmatize(abstract, nlp):
    '''
    
    '''
    doc = nlp(abstract)
    lemmas = [tk.lemma_ for tk in doc]
    return lemmas

results = []
for idx, row in tqdm(metadata_df.iterrows()):
    l = lemmatize(row['abstract'], nlp)
    results.append(l)
    

metadata_df['lemmas'] = results
metadata_df

In [ ]:
# Tokenize and lemmatize the data
# Split the data to avoid memory problems
idx_ls = list(range(0, len(metadata_df), 10000))
for i, ii in enumerate(idx_ls):
    if i < len(idx_ls) - 1:
        idx1 = idx_ls[i]
        idx2 = idx_ls[i + 1] 
    else:
        idx1 = idx_ls[i]
        idx2 = len(metadata_df) 

    print('{} to {}'.format(idx1, idx2))
        
    metadata_df_aux = metadata_df.iloc[idx1:idx2].copy()
    metadata_df_aux['tokens'] = metadata_df_aux['abstract'].progress_apply(nlp)
    metadata_df_aux['lemmas'] = metadata_df_aux['tokens'].progress_apply(lambda token: [tk.lemma_ for tk in token])
    metadata_df_aux.to_csv('./2021-04-26/metadata_processed_{}_{}.csv'.format(idx1, idx2), index=False)
    del metadata_df_aux
    gc.collect()

In [ ]:
metadata_df.to_csv('./2021-04-26/metadata_processed.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Save the preprocessed data
SAVE_PATH = '/content/gdrive/MyDrive/Master in Information Health Engineering/Speech & Natural Language Processing/Labs/FINAL PROJECT'
metadata_df.to_csv(SAVE_PATH + '/metadata_processed.csv', index=False)

nlp.to_disk(SAVE_PATH + '/en_core_extended')

In [ ]:
doc = nlp(metadata_df['abstract'].iloc[0])
doc

In [ ]:
[d for d in doc]

In [ ]:
print(colored('============= Original Text =============', 'blue'))
print(doc)
print(colored('\n============= Lemmatized Text =============', 'red'))
print(' '.join([tk.lemma_ for tk in doc]))
print(colored('\n============= Entities Found =============', 'green'))
print('\n'.join([ent.text for ent in doc.ents]))

In [ ]:
!pip install crossrefapi==1.0.3

In [ ]:
from crossref.restful import Works

works = Works()
info = works.doi('10.1038/s41591-020-1132-9')

info

In [ ]:
for idx, row in data.iterrows():
    info = works.doi(row['doi'])
    print([a['affiliation'] for a in info['author']])